# Population vs. Median Home Prices
#### *Linear Regression with Single Variable*

*Note, this notebook requires Spark 1.6+*

In [3]:
%scala if (org.apache.spark.BuildInfo.sparkBranch < "1.6") sys.error("Attach this notebook to a cluster running Spark 1.6+")

### Load and parse the data

In [5]:
# Use the Spark CSV datasource with options specifying:
#  - First line of file is a header
#  - Automatically infer the schema of the data
data = sqlContext.read.format("com.databricks.spark.csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/databricks-datasets/samples/population-vs-price/data_geo.csv")
data.cache()  # Cache data for faster reuse
data.count()

Out[ 1 ]: 294

In [6]:
display(data)

2014 rank,City,State,State Code,2014 Population estimate,2015 median sales price
101.0,Birmingham,Alabama,AL,212247.0,162.9
125.0,Huntsville,Alabama,AL,188226.0,157.7
122.0,Mobile,Alabama,AL,194675.0,122.5
114.0,Montgomery,Alabama,AL,200481.0,129.0
64.0,Anchorage[19],Alaska,AK,301010.0,List()
78.0,Chandler,Arizona,AZ,254276.0,List()
86.0,Gilbert[20],Arizona,AZ,239277.0,List()
88.0,Glendale,Arizona,AZ,237517.0,List()
38.0,Mesa,Arizona,AZ,464704.0,List()
148.0,Peoria,Arizona,AZ,166934.0,List()


In [7]:
data = data.dropna()  # drop rows with missing values
data.count()

Out[ 3 ]: 109

In [8]:
# This will let us access the table from our SQL notebook!
data.registerTempTable("data_geo")

## Limit data to Population vs. Price
(for our ML example)

We also use `LabeledPoint` to convert the feature (population) to a Vector type, to prep the data for ML algorithms.

In [10]:
from pyspark.mllib.regression import LabeledPoint  # convenience for specifying schema
data = data.select("2014 Population estimate", "2015 median sales price")\
  .map(lambda r: LabeledPoint(r[1], [r[0]]))\
  .toDF()
display(data)

features,label
"List(1.0, 1.0, List(), List(212247.0))",162.9
"List(1.0, 1.0, List(), List(188226.0))",157.7
"List(1.0, 1.0, List(), List(194675.0))",122.5
"List(1.0, 1.0, List(), List(200481.0))",129.0
"List(1.0, 1.0, List(), List(1537058.0))",206.1
"List(1.0, 1.0, List(), List(527972.0))",178.1
"List(1.0, 1.0, List(), List(197706.0))",131.8
"List(1.0, 1.0, List(), List(346997.0))",685.7
"List(1.0, 1.0, List(), List(3928864.0))",434.7
"List(1.0, 1.0, List(), List(319504.0))",281.0


## Scatterplot of the data using ggplot

In [12]:
import numpy as np
import matplotlib.pyplot as plt

x = data.map(lambda p: (p.features[0])).collect()
y = data.map(lambda p: (p.label)).collect()

from pandas import *
from ggplot import *
pydf = DataFrame({'pop':x,'price':y})
p = ggplot(pydf, aes('pop','price')) + \
    geom_point(color='blue') 
display(p)

## Linear Regression

**Goal**
* Predict y = 2015 Median Housing Price
* Using feature x = 2014 Population Estimate

**References**
* [MLlib LinearRegression user guide](http://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression)
* [PySpark LinearRegression API](http://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.regression.LinearRegression)

In [14]:
# Import LinearRegression class
from pyspark.ml.regression import LinearRegression

In [15]:
# Define LinearRegression algorithm
lr = LinearRegression()

In [16]:
# Fit 2 models, using different regularization parameters
modelA = lr.fit(data, {lr.regParam:0.0})
modelB = lr.fit(data, {lr.regParam:100.0})

In [17]:
print ">>>> ModelA intercept: %r, coefficient: %r" % (modelA.intercept, modelA.coefficients[0])

>>>> ModelA intercept: 191.29427575139374, coefficient: 3.779789682338261e-05

In [18]:
print ">>>> ModelB intercept: %r, coefficient: %r" % (modelB.intercept, modelB.coefficients[0])

>>>> ModelB intercept: 199.85112564667136, coefficient: 2.1603499483717248e-05

## Make predictions

Calling `transform()` on data adds a new column of predictions.

In [20]:
# Make predictions
predictionsA = modelA.transform(data)
display(predictionsA)

features,label,prediction
"List(1.0, 1.0, List(), List(212247.0))",162.9,199.31676595846622
"List(1.0, 1.0, List(), List(188226.0))",157.7,198.40882267887176
"List(1.0, 1.0, List(), List(194675.0))",122.5,198.65258131548575
"List(1.0, 1.0, List(), List(200481.0))",129.0,198.8720359044423
"List(1.0, 1.0, List(), List(1537058.0))",206.1,249.39183544694856
"List(1.0, 1.0, List(), List(527972.0))",178.1,211.2505069330287
"List(1.0, 1.0, List(), List(197706.0))",131.8,198.76714674075743
"List(1.0, 1.0, List(), List(346997.0))",685.7,204.41003255541705
"List(1.0, 1.0, List(), List(3928864.0))",434.7,339.797071856496
"List(1.0, 1.0, List(), List(319504.0))",281.0,203.37085497805177


## Evaluate the Model
#### Predicted vs. True label

In [22]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="rmse")
RMSE = evaluator.evaluate(predictionsA)
print("ModelA: Root Mean Squared Error = " + str(RMSE))

ModelA: Root Mean Squared Error = 128.602026843

In [23]:
predictionsB = modelB.transform(data)
RMSE = evaluator.evaluate(predictionsB)
print("ModelB: Root Mean Squared Error = " + str(RMSE))

ModelB: Root Mean Squared Error = 129.496300193

# Linear Regression Plots

In [25]:
import numpy as np
from pandas import *
from ggplot import *

pop = data.map(lambda p: (p.features[0])).collect()
price = data.map(lambda p: (p.label)).collect()
predA = predictionsA.select("prediction").map(lambda r: r[0]).collect()
predB = predictionsB.select("prediction").map(lambda r: r[0]).collect()

pydf = DataFrame({'pop':pop,'price':price,'predA':predA, 'predB':predB})

## View the Python Pandas DataFrame (pydf)

In [27]:
pydf

Out[ 16 ]: 
 pop predA predB price
0 212247 199.316766 204.436404 162.9
1 188226 198.408823 203.917466 157.7
2 194675 198.652581 204.056787 122.5
3 200481 198.872036 204.182217 129.0
4 1537058 249.391835 233.056957 206.1
5 527972 211.250507 211.257168 178.1
6 197706 198.767147 204.122267 131.8
7 346997 204.410033 207.347475 685.7
8 3928864 339.797072 284.728337 434.7
9 319504 203.370855 206.753530 281.0
10 485199 209.633777 210.333122 275.8
11 1381069 243.495779 229.687049 510.3
12 852469 223.515811 218.267439 748.3
13 1015785 229.688812 221.795636 900.0
14 105112 195.267288 202.121913 442.2
15 445830 208.145712 209.482614 220.1
16 663862 216.386863 214.192868 338.1
17 147612 196.873699 203.040061 342.7
18 124705 196.007862 202.545190 202.6
19 130282 196.218661 202.665673 205.1
20 658893 216.199045 214.085520 367.8
21 169854 197.714400 203.520566 208.0
22 128460 196.149794 202.626311 169.9
23 853382 223.550321 218.287163 182.5
24 102346 195.162739 202.062157 136.0
25 430332 207.559920 209.147803 269.1
26 105838 195.294730 202.137597 145.0
27 174110 197.875268 203.612511 172.0
28 188107 198.404325 203.914895 170.5
29 358699 204.852344 207.600279 156.0
.. ... ... ... ...
79 141003 196.623893 202.897284 107.2
80 281031 201.916656 205.922379 83.8
81 399682 206.401415 208.485656 145.1
82 160561 197.363144 203.319805 211.3
83 619360 214.704781 213.231469 289.4
84 161637 197.403814 203.343050 193.9
85 119104 195.796156 202.424189 162.7
86 1560297 250.270221 233.559001 204.9
87 179154 198.065920 203.721479 233.3
88 168586 197.666472 203.493173 159.5
89 173778 197.862719 203.605339 143.6
90 184281 198.259710 203.832240 149.2
91 656861 216.122240 214.041622 132.1
92 644014 215.636650 213.764082 186.4
93 120958 195.866234 202.464242 145.7
94 197254 198.750062 204.112502 144.4
95 912791 225.795856 219.570606 249.1
96 117585 195.738741 202.391373 133.3
97 320434 203.406007 206.773621 172.9
98 1281047 239.715158 227.526224 192.5
99 679036 216.960408 214.520680 138.4
100 2239558 275.944858 248.233416 200.3
101 1436697 245.598401 230.888809 184.7
102 190884 198.509289 203.974888 243.3
103 217853 199.528661 204.557513 212.5
104 450980 208.340371 209.593872 190.0
105 668342 216.556198 214.289652 352.4
106 212052 199.309395 204.432191 180.2
107 245691 200.580879 205.158911 226.2
108 599642 213.959482 212.805491 195.4

[109 rows x 4 columns]

## ggplot figure
Now that the Python Pandas DataFrame (pydf), use ggplot and display the scatterplot and the two regression models

In [29]:
p = ggplot(pydf, aes('pop','price')) + \
    geom_point(color='blue') + \
    geom_line(pydf, aes('pop','predA'), color='red') + \
    geom_line(pydf, aes('pop','predB'), color='green') + \
    scale_x_log10() + scale_y_log10()
display(p)